# Visualize GNoME Crystal Structures

The stable crystals released with the [GNoME](https://www.nature.com/articles/s41586-023-06735-9) paper cover a broad range of the periodic table and exhibit many new prototypes. In this colab, we showcase how the provided dataset can be used with various visualization utilities, allowing researchers to browse the dataset and explore the provided crystal structures.

# Install Visualization Libraries

In [ ]:
#@markdown Installation may require restarting the current colab kernel.
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!pip install vtk matplotlib x3d vapory pymatgen ase
!mamba install -c conda-forge nglview ipywidgets=7

In [ ]:
from typing import Tuple

import os
import pandas as pd
import shutil
import tempfile
import zipfile

import pymatgen as mg
import pymatgen.io.cif

import ase.io.cif
import ase.visualize
import ase

# Enable custom widgets in colab
from google.colab import output
output.enable_custom_widget_manager()

# Download the Dataset

In [ ]:
PUBLIC_LINK = "https://storage.googleapis.com/"
BUCKET_NAME = "gdm_materials_discovery"

FOLDER_NAME = "gnome_data"
FILES = (
    "stable_materials_summary.csv",
    "by_reduced_formula.zip",
)

def download_from_link(link: str, output_dir: str):
  """Download a file from a public link using wget."""
  os.system(f"wget {link} -P {output_dir}")

parent_directory = os.path.join(PUBLIC_LINK, BUCKET_NAME)
for filename in FILES:
  public_link = os.path.join(parent_directory, FOLDER_NAME, filename)
  download_from_link(public_link, '.')

# Read the contents of the zipfile
# Due to the size of the release, this takes about 30 seconds but prevents
# the need to extract all files in order to read individual structures
z = zipfile.ZipFile('by_reduced_formula.zip')

# Choose A Crystal Structure

In [ ]:
# Parse the summary csv
gnome_crystals = pd.read_csv('stable_materials_summary.csv', index_col=0)
gnome_crystals

In [ ]:
def obtain_structure(
    reduced_formula: str | None = None,
) -> Tuple[ase.Atoms, mg.core.Structure]:
  """Obtain the structure from a provided reduced formula."""
  temp_dir = tempfile.TemporaryDirectory()
  extension = f"{reduced_formula}.CIF"
  temp_path = os.path.join(temp_dir.name, extension)

  with z.open(os.path.join('by_reduced_formula', extension)) as zf:
    with open(temp_path, 'wb') as fp:
      shutil.copyfileobj(zf, fp)

  atoms = ase.io.read(temp_path)
  structure = mg.core.Structure.from_file(temp_path)
  temp_dir.cleanup()
  return atoms, structure

In [ ]:
# @title Provide a reduced formula to select from the GNoME dataset.
# @markdown For a random structure, set the reduced formula to 'random' \\
# @markdown Make sure to run this cell to select this select the formula.

reduced_formula = 'random' # @param {type:"string"}

if reduced_formula == 'random':
  sample = gnome_crystals.sample()
  reduced_formula = sample['Reduced Formula'].item()
else:
  # Ensure that composition entries are converted to reduced formulas
  reduced_formula = mg.core.Composition(reduced_formula).reduced_formula
  sample = gnome_crystals[
      gnome_crystals['Reduced Formula'] == reduced_formula].iloc[0:1]

atoms, structure = obtain_structure(reduced_formula)

In [ ]:
sample

## Example \#1: X3D via ASE

A simplified viewer based on X3D is provided by [ASE](https://wiki.fysik.dtu.dk/ase/) and provides visualization capabilities with minimal addtional dependencies.


In [ ]:
ase.visualize.view(atoms, viewer='x3d')

## Example \#2: nglview via ASE

nglview provides tooling to interactively view molcular structures, with significant flexibility including colors of the associated atoms. Notably, this tooling relies on ipywidgets (where the latest version 8 often fails to display
in colab). We have fixed this by using ipywidgets 7 in the installation abvoe
but note that adjustments might cause this tooling to not be displayed.

In [ ]:
ase.visualize.view(atoms, viewer='ngl')

# Example \#3: Matplotlib via ASE

Finally, due to the prevalence of matplotlib, we also use ASE utilities to provide a strategy for creating figures from the associated crystal structure.

In [ ]:
from ase.visualize.plot import plot_atoms
plot_atoms(atoms)

Additional tooling for crystals structures is made available through libraries such as Pymatgen (in the pymatgen.vis library). We included the above 3 examples due to compatability with colabs, but if anyone wants to include additional visualization examples, please feel free to extend this colab.

# Download files for local visualization:

Finally, we provide a mechanism for downloading various formats of the chosen structure. Please adjust the desired format via the drop down bar on the right
and run the cell below to download the associated file.

In [ ]:
from google.colab import files
import tempfile
import os
from tempfile import mkstemp

fmt = "cif" #@param ['cif', 'poscar', 'cssr', 'json', 'yaml', 'yml', 'xsf', 'mcsqs', 'res', 'pwmat']
temp_dir = tempfile.TemporaryDirectory()
filename = os.path.join(temp_dir.name, f"{structure.reduced_formula}.{fmt}")
structure.to(filename, fmt=fmt)
files.download(filename)
temp_dir.cleanup()